In [113]:
import numpy as np

In [121]:
def write_header_lines(file, layer_height, line_width, layer_count, mesh, speed=1200):
    with open(file, 'w') as f:
        header_lines = [
            f"layer_height = {layer_height}",
            f"\nline_width = {line_width}",
            f"\nlayer_count = {layer_count}",
            f"\nmesh = {mesh}"
        ]
        
        initialize_lines = [
            "\nG21 ;start of the code",
            "\nG1 Z15 F300",
            "\nG28 X0 Y0 ;Home",
            "\nG92 X0 Y0 ;Consider this as current",
            "\nG0 X50 Y50 F3000 ;Go-to Offset",  
            "\nG92 X0 Y0 ;Reset",
            "\n",
            "\nG0 F3600 X5.750 Y5.750 Z0.3",
            "\nM7",
            f"\nF{speed}"
            "\n\n"
        ]
        
        f.writelines(header_lines)
        f.writelines(initialize_lines)
        
        
def writ_finish_lines(file):
    with open(file, 'a') as f:
        finish_lines = [
            "\n\n;Finish",
            "\nM9",
            "\nG92 E1"
            "\nG1 E-1 F300"
            "\nG28 ;Home",
        ]
        f.writelines(finish_lines)

In [115]:
def layer_rentry(file, s, w, start_xy_sign, start_z, layer_height, wall_layer_count, clock_wise):
    """
    Args:
        s:   side length of the square
        w:   line width
    """  
    s = s - w
    lines = []
    xy = np.array([s/2*start_xy_sign[0], s/2*start_xy_sign[1]])
    z = start_z + layer_height
    
    if clock_wise:
        if start_xy_sign == [1,1]:
            # wall 1
            for i in range (wall_layer_count):
                if i == 0:
                    xy += [0, -(s-w*i*2)]
                else:
                    xy += [0, -(s-w*i*2+w)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [-(s-w*i*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, (s-w*i*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [(s-w*i*2-w), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            if wall_layer_count != 0:
                xy += [0, -w]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # infill 1
            xy += [0, -(s-w*wall_layer_count*2)]
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            for i in range (int(s/w-wall_layer_count*2)):
                xy += [-w, 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                if (i % 2):
                    xy += [0, -(s-w*wall_layer_count*2)]
                else:
                    xy += [0, (s-w*wall_layer_count*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            
            # go to layer 2
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]} Z{z}")
            
            # infill 2
            for i in range (int(s/w-wall_layer_count*2)):
                if (i % 2):
                    xy += [-(s-w*wall_layer_count*2), 0]
                else:
                    xy += [(s-w*wall_layer_count*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, -w]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            xy += [-(s-w*wall_layer_count*2), 0]
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            
            # wall 2
            if wall_layer_count != 0:
                xy += [-w, 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            for i in range (wall_layer_count):
                xy += [0, (s-w*(wall_layer_count-i-1)*2-w)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [(s-w*(wall_layer_count-i-1)*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, -(s-w*(wall_layer_count-i-1)*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                if (i+1 == wall_layer_count):
                    xy += [-(s-w*(wall_layer_count-i-1)*2), 0]
                else:
                    xy += [-(s-w*(wall_layer_count-i-1)*2+w), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # go to layer 3
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]} Z{z+layer_height}")


        elif start_xy_sign == [-1,-1]:
            # wall 1
            for i in range (wall_layer_count):
                if i == 0:
                    xy += [0, (s-w*i*2)]
                else:
                    xy += [0, (s-w*i*2+w)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [(s-w*i*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, -(s-w*i*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [-(s-w*i*2-w), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            if wall_layer_count != 0:
                xy += [0, w]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # infill 1
            xy += [0, (s-w*wall_layer_count*2)]
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            for i in range (int(s/w-wall_layer_count*2)):
                xy += [w, 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                if (i % 2):
                    xy += [0, (s-w*wall_layer_count*2)]
                else:
                    xy += [0, -(s-w*wall_layer_count*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # go to layer 2
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]} Z{z}")
            
            # infill 2
            for i in range (int(s/w-wall_layer_count*2)):
                if (i % 2):
                    xy += [(s-w*wall_layer_count*2), 0]
                else:
                    xy += [-(s-w*wall_layer_count*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, w]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            xy += [(s-w*wall_layer_count*2), 0]
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # wall 2
            if wall_layer_count != 0:
                xy += [w, 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            for i in range (wall_layer_count):
                xy += [0, -(s-w*(wall_layer_count-i-1)*2-w)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [-(s-w*(wall_layer_count-i-1)*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, (s-w*(wall_layer_count-i-1)*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                if (i+1 == wall_layer_count):
                    xy += [(s-w*(wall_layer_count-i-1)*2), 0]
                else:
                    xy += [(s-w*(wall_layer_count-i-1)*2+w), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # go to layer 3
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]} Z{z+layer_height}")
            
    else:
        if start_xy_sign == [1,1]:
            # wall 1
            for i in range (wall_layer_count):
                if i == 0:
                    xy += [-(s-w*i*2), 0]
                else:
                    xy += [-(s-w*i*2+w), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, -(s-w*i*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [(s-w*i*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, (s-w*i*2-w)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            if wall_layer_count != 0:
                xy += [-w,0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # infill 1
            xy += [-(s-w*wall_layer_count*2), 0]
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            for i in range (int(s/w-wall_layer_count*2)):
                xy += [0, -w]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                if (i % 2):
                    xy += [-(s-w*wall_layer_count*2), 0]
                else:
                    xy += [(s-w*wall_layer_count*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # go to layer 2
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]} Z{z}")
            
            # infill 2
            for i in range (int(s/w-wall_layer_count*2)):
                if (i % 2):
                    xy += [0, -(s-w*wall_layer_count*2)]
                else:
                    xy += [0, (s-w*wall_layer_count*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [-w, 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            xy += [0, -(s-w*wall_layer_count*2)]
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # wall 2
            if wall_layer_count != 0:
                xy += [0, -w]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            for i in range (wall_layer_count):
                xy += [(s-w*(wall_layer_count-i-1)*2-w), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, (s-w*(wall_layer_count-i-1)*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [-(s-w*(wall_layer_count-i-1)*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                if (i+1 == wall_layer_count):
                    xy += [0, -(s-w*(wall_layer_count-i-1)*2)]
                else:
                    xy += [0, -(s-w*(wall_layer_count-i-1)*2+w)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # go to layer 3
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]} Z{z+layer_height}")
            
        elif start_xy_sign == [-1,-1]:
            # wall 1
            for i in range (wall_layer_count):
                if i == 0:
                    xy += [(s-w*i*2), 0]
                else:
                    xy += [(s-w*i*2+w), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, (s-w*i*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [-(s-w*i*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, -(s-w*i*2-w)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            if wall_layer_count != 0:
                xy += [w, 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # infill 1
            xy += [(s-w*wall_layer_count*2), 0]
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            for i in range (int(s/w-wall_layer_count*2)):
                xy += [0, w]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                if (i % 2):
                    xy += [(s-w*wall_layer_count*2), 0]
                else:
                    xy += [-(s-w*wall_layer_count*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # go to layer 2
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]} Z{z}")
            
            # infill 2
            for i in range (int(s/w-wall_layer_count*2)):
                if (i % 2):
                    xy += [0, (s-w*wall_layer_count*2)]
                else:
                    xy += [0, -(s-w*wall_layer_count*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [w, 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            xy += [0, (s-w*wall_layer_count*2)]
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # wall 2
            if wall_layer_count != 0:
                xy += [0, w]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
            for i in range (wall_layer_count):
                xy += [-(s-w*(wall_layer_count-i-1)*2-w), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [0, -(s-w*(wall_layer_count-i-1)*2)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                xy += [(s-w*(wall_layer_count-i-1)*2), 0]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                if (i+1 == wall_layer_count):
                    xy += [0, (s-w*(wall_layer_count-i-1)*2)]
                else:
                    xy += [0, (s-w*(wall_layer_count-i-1)*2+w)]
                lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
                
            # go to layer 3
            lines.append(f"\nG1 X{xy[0]} Y{xy[1]} Z{z+layer_height}")
            
    with open(file, 'a') as f:
        f.writelines(lines)

In [123]:
def write_G1_line(delta_x, delta_y, xy, lines):
    xy += [delta_x, delta_y]
    lines.append(f"\nG1 X{xy[0]} Y{xy[1]}")
    return xy, lines

def go_to_next_layer(z, xy, lines, layer_height):
    z += layer_height
    lines.append(f"\nG1 X{xy[0]} Y{xy[1]} Z{z}")
    return z, lines

In [124]:
def layer_wall(file, s, w, z, layer_height, clock_wise):
    """
    Args:
        s:   side length of the square
        w:   line width
    """  
    s = s - w
    lines = []
    xy = np.array([s/2, s/2])
    
    if clock_wise:
        xy, lines = write_G1_line(0, -s, xy, lines)
        xy, lines = write_G1_line(-s, 0, xy, lines)
        xy, lines = write_G1_line(0, s, xy, lines)
        xy, lines = write_G1_line(s, 0, xy, lines)
        z, lines = go_to_next_layer(z, xy, lines, layer_height)
    
    else: 
        xy, lines = write_G1_line(-s, 0, xy, lines)
        xy, lines = write_G1_line(0, -s, xy, lines)
        xy, lines = write_G1_line(s, 0, xy, lines)
        xy, lines = write_G1_line(0, s, xy, lines)
        z, lines = go_to_next_layer(z, xy, lines, layer_height)
        
    with open(file, 'a') as f:
        f.writelines(lines)
    
    return z

In [137]:
speed = 1800
file = '7.17//speed_test_noreturn'+str(speed)+'.gcode'
layer_height = 0.4
line_width = 0.5
layer_count = 15
mesh = 'only wall'
side_length = 12
wall_layer_count = 0
write_header_lines(file, layer_height, 0.4, layer_count, mesh, speed)

z = 0.3
for i in range (layer_count):
    z = layer_wall(file, side_length, line_width, z, layer_height, True)
    # if i % 2:
    #     z = layer_wall(file, side_length, line_width, z, layer_height, True)
    # else:
    #     z = layer_wall(file, side_length, line_width, z, layer_height, False)
        
writ_finish_lines(file)

In [112]:
file = '7.16//rentry_0.gcode'
layer_height = 0.4
line_width = 0.5
layer_count = 30
mesh = 'platform1315.stl'
side_length = 12
wall_layer_count = 0

write_header_lines(file, layer_height, line_width, layer_count, mesh)
z = 0.3
for i in range (int(12/layer_height/8)):
    layer_rentry(file, side_length, line_width, [1,1], z, layer_height, wall_layer_count, False)
    z += 0.8
    layer_rentry(file, side_length, line_width, [-1,-1], z, layer_height, wall_layer_count, False)
    z += 0.8
    layer_rentry(file, side_length, line_width, [1,1], z, layer_height, wall_layer_count, False)
    z += 0.8
    layer_rentry(file, side_length, line_width, [-1,-1], z, layer_height, wall_layer_count, False)
    z += 0.8
layer_rentry(file, side_length, line_width, [1,1], z, layer_height, wall_layer_count, False)
z += 0.8
layer_rentry(file, side_length, line_width, [-1,-1], z, layer_height, wall_layer_count, False)
z += 0.8
layer_rentry(file, side_length, line_width, [1,1], z, layer_height, wall_layer_count, False)
z += 0.8

In [106]:
z

12.300000000000002

In [68]:
12/layer_height/8

3.75